In [1]:
import h3
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('dadosSP.csv',delimiter=';')
dfCoordenates=pd.read_csv('coordenadas.csv',delimiter=';')
dfCoordenates['Station']=df.Station.unique()

In [3]:
dfCoordenates.set_index('Station',inplace=True)
dfCoordenates['h3id'] = dfCoordenates.apply(lambda row: h3.geo_to_h3(row.Latitude, row.Longitude, 9), axis=1)

h3dict = dfCoordenates.h3id.to_dict()

In [4]:
# join date and hour in dataframe df
df['Date']=pd.to_datetime(df.Date) + pd.to_timedelta(df.Hour)
df.drop(columns=['Hour'],inplace=True)

# convert other columns to numeric
df[df.columns[2:]] = df[df.columns[2:]].apply(pd.to_numeric, errors='coerce', axis=1)

### April 2019

In [5]:
# April 2019
i = pd.date_range('2019-04-01 00:00:00', periods=31, freq='24H')
dfTemp=df.loc[(df.Date >= i[0]) & (df.Date <= i[-1])].copy()
# indexes for 2019-05-01
temp=dfTemp.loc[dfTemp['Date']==pd.date_range('2019-05-01 00:00:00', periods=1, freq='24H')[0]].index
# Remove 2019-05-01
dfTemp.drop(temp,inplace=True)
# Reset index
dfTemp.reset_index(inplace=True,drop=True)

In [6]:
dfTemp['h3id'] = dfTemp.Station.apply(lambda s: h3dict[s])
dfTemp['NO2']=dfTemp['NO2']/1.88
dfTemp['NO']=dfTemp['NO']/1.25
dfTemp['time'] = pd.to_datetime(dfTemp.Date)

In [10]:
dfTemp[['h3id', 'time', 'NO2', 'NO']].to_csv('y_data.csv', index=False)